# Tarea 1 - Pregunta 3

## Estrategia
A partir de la lista de strings de input se genera un árbol MerkleTree, el cual almacena una raíz que es de clase Node. Dicha raíz está asociada a un nodo izquierdo y derecho, que representan a sus hijos en el árbol, además, cada nodo guarda la información del valor almacenado, es decir, del hash acumulado que viene desde los hijos.

La construcción del árbol se hace a partir de las hojas del fondo, es decir, bottom-up, se tiene inicialmente una lista de nodos con los string de input con el hash aplicado. Luego, se crea una nueva lista de nodos tomando cada string hasheado de par en par obteniendo un solo padre por cada 2 nodos. Y así hacia arriba hasta llegar a la raíz. En caso de no tener un número par de nodos en algún nivel, se rellena con el nodo hermano.

Para obtener la prueba de algún elemento, se busca dicho elemento en el árbol con DFS, se obtiene su hermano y luego se recorre hacia arriba buscando los nodos tíos que forman la prueba, esto se ve en la función get_node de la clase Node.

Finalmente, verify simplemente recorre la lista obtenida por la función get_proof_for y va aplicando la función de hash de manera incremental hasta llegar al último tío. Retorna true si el hash obtenido es igual al de la raíz.

## Solución

In [8]:
from typing import Callable

def _hash_func(string: str) -> str:
    return string

class Node:
    def __init__(self, value):
        self.left = None
        self.right = None
        self.parent = None
        self.value = value

    def get_node(self, item: str):
        """
        DFS hasta encontrar el nodo con el valor solicitado
        """
        if self.value == item:
            return self
        if self.left: 
            left = self.left.get_node(item)
            if left and left.value == item:
                return left
        if self.right: 
            right = self.right.get_node(item)
            if right and right.value == item:
                return right
        return None

class MerkleTree:
    def __init__(self, strings, hash_func):
        """
        Arguments:
            strings: The set of strings S
        """
        self.strings = strings
        self.hash_func = hash_func
        self.root = None


    def get_root(self) -> str:
        """
        Returns:
            root: Root of the Merkle Tree
        """
        nodes = list()

        for i in self.strings:
            nodes.append(Node(i))

        while (len(nodes) > 1):
            parents = []
            for i in range(0, len(nodes), 2):
                if i == len(nodes) - 1 and len(nodes) % 2 != 0:
                    nodes.append(nodes[i])
                parent = Node(self.hash_func(nodes[i].value + nodes[i + 1].value))
                parent.left = nodes[i]
                parent.right = nodes[i + 1]
                self.insert_parent(parent)
                parents.append(parent)
            nodes = parents
        self.root = nodes[0]
        self.value = self.root.value
        return self.value
            
    def insert_parent(self, node):
        node.left.parent = node
        node.right.parent = node

    def get_node(self, item: str):
        return self.root.get_node(item)

    def get_proof_for(self, item: str) -> None or list:
        """
        Returns:
            result: None if the item is not part of the leafs of the tree
            A list with the necessary info to prove that the
            item is part of the leafs of the tree
        """
        item_node = self.get_node(item)
        proof = list()

        if item_node.left != None or item_node.right != None: return None

        current_node = item_node

        while current_node.parent != None:
            value, dir = (current_node.parent.right.value, "d") if current_node.parent.left == current_node else (current_node.parent.left.value, "i")
            proof.append((value, dir))
            current_node = current_node.parent

        return proof

def verify(root: str, item: str, proof: list([(str, "d" or "i")]), hash_func: Callable[[str], str]) -> bool:
    """
    Arguments:
        root: The root of a merkle tree
        item: An arbitrary string
        proof: An alleged proof that item is part of a Merkle tree with root root
        hash_func: An arbitrary hash function
    Returns:
        correct: whether the proof is coorect or not
    """
    
    hash = item
    for value, pos in proof: hash = hash_func(hash + value) if pos == "d" else hash_func(value + hash)
    return hash == root
    

strings = ["a", "b", "c", "d", "e", "f"]
tree = MerkleTree(strings, _hash_func)
root = tree.get_root()
proof = tree.get_proof_for("d")
print(verify(root, "d", proof, _hash_func))


True
